# Detecting Payment Card Fraud

In this section, we'll look at a credit card fraud detection dataset, and build a binary classification model that can identify transactions as either fraudulent or valid, based on provided, *historical* data. In a [2016 study](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf), it was estimated that credit card fraud was responsible for over 20 billion dollars in loss, worldwide. Accurately detecting cases of fraud is an ongoing area of research.

<img src=notebook_ims/fraud_detection.png width=50% />

### Labeled Data

The payment fraud data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data). This has features and labels for thousands of credit card transactions, each of which is labeled as fraudulent or valid. In this notebook, we'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.

### Binary Classification

Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data into one of our two transaction classes: fraudulent or valid.  We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

### Making Improvements

A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than fraudulent).

---

First, import the usual resources.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

I'm storing my **SageMaker variables** in the next cell:
* sagemaker_session: The SageMaker session we'll use for training models.
* bucket: The name of the default S3 bucket that we'll use for data storage.
* role: The IAM role that defines our data and model permissions.

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## Loading and Exploring the Data

Next, I am loading the data and unzipping the data in the file `creditcardfraud.zip`. This directory will hold one csv file of all the transaction data, `creditcard.csv`.

As in previous notebooks, it's important to look at the distribution of data since this will inform how we develop a fraud detection model. We'll want to know: How many data points we have to work with, the number and type of features, and finally, the distribution of data over the classes (valid or fraudulent).

In [3]:
# only have to run once
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud


--2021-07-10 09:44:56--  https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.78.230
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.78.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip’

creditcardfraud.zip 100%[===================>]  65.95M  38.1MB/s    in 1.7s    

2021-07-10 09:44:58 (38.1 MB/s) - ‘creditcardfraud.zip’ saved [69155632/69155632]

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [4]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### EXERCISE: Calculate the percentage of fraudulent data

Take a look at the distribution of this transaction data over the classes, valid and fraudulent. 

Complete the function `fraudulent_percentage`, below. Count up the number of data points in each class and calculate the *percentage* of the data points that are fraudulent.

In [8]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    
    # your code here
    counts = transaction_df['Class'].value_counts()
    valid_counts = counts[0]
    fraud_counts = counts[1]
    fraud_percentage = fraud_counts / (valid_counts + fraud_counts)
    
    return fraud_percentage


Test out your code by calling your function and printing the result.

In [9]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


### EXERCISE: Split into train/test datasets

In this example, we'll want to evaluate the performance of a fraud classifier; training it on some training data and testing it on *test data* that it did not see during the training process. So, we'll need to split the data into separate training and test sets.

Complete the `train_test_split` function, below. This function should:
* Shuffle the transaction data, randomly
* Split it into two sets according to the parameter `train_frac`
* Get train/test features and labels
* Return the tuples: (train_features, train_labels), (test_features, test_labels)

In [10]:
# split into train/test
def split_data(transaction_df, test_size= 0.3, random_state=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: train_features, train_labels test_features, test_labels
       '''
    
    # shuffle and split the data
    train_df, test_df = train_test_split(transaction_df, test_size=test_size, random_state = random_state)
    
    train_features = train_df.drop(['Class'], axis = 1)
    train_labels = train_df['Class']
    test_features = test_df.drop(['Class'], axis = 1)
    test_labels = test_df['Class']
    
    return train_features, test_features, train_labels, test_labels


### Test Cell

In the cells below, I'm creating the train/test data and checking to see that result makes sense. The tests below test that the above function splits the data into the expected number of points and that the labels are indeed, class labels (0, 1).

In [11]:
# get train/test data
train_features, test_features, train_labels, test_labels = split_data(transaction_df, test_size=0.3, random_state=1)

In [12]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features.iloc[:,0])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  199364
Test data pts:  85443

First item: 
 191125    129124.0
153710     99901.0
261216    159917.0
190724    128961.0
127492     78349.0
            ...   
21440      31666.0
117583     74729.0
73349      55095.0
267336    162728.0
128037     78576.0
Name: Time, Length: 199364, dtype: float64
Label:  0

Tests passed!


---
# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

In this notebook, you'll define and train the SageMaker, built-in algorithm, [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html). 

A LinearLearner has two main applications:
1. For regression tasks in which a linear line is fit to some data points, and you want to produce a predicted output value given some data point (example: predicting house prices given square area).
2. For binary classification, in which a line is separating two classes of data and effectively outputs labels; either 1 for data that falls above the line or 0 for points that fall on or below the line.

<img src='notebook_ims/linear_separator.png' width=50% />

In this case, we'll be using it for case 2, and we'll train it to separate data into our two classes: valid or fraudulent. 

### EXERCISE: Create a LinearLearner Estimator

You've had some practice instantiating built-in models in SageMaker. All estimators require some constructor arguments to be passed in. See if you can complete this task, instantiating a LinearLearner estimator, using only the [LinearLearner documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html) as a resource. This takes in a lot of arguments, but not all are required. My suggestion is to start with a simple model, utilizing default values where applicable. Later, we will discuss some specific hyperparameters and their use cases.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [13]:
# import LinearLearner
from sagemaker import LinearLearner

# # sagemaker session, role
# sagemaker_session = sagemaker.Session()
# role = sagemaker.get_execution_role()

# # S3 bucket name
# bucket = sagemaker_session.default_bucket()

# ouput
prefix = 'creditcard'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner
linear = LinearLearner(role = role,
                       train_instance_count = 1, 
                       train_instance_type = 'ml.c4.xlarge',
                       predictor_type = 'binary_classifier',
                       output_path = output_path,
                       sagemaker_session = sagemaker_session,
                       epochs = 10)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### EXERCISE: Convert data into a RecordSet format

Next, prepare the data for a built-in model by converting the train features and labels into numpy array's of float values. Then you can use the [record_set function](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set) to format the data as a RecordSet and prepare it for training!

In [14]:
# train_features, test_features, train_labels, test_labels
type(train_features)

pandas.core.frame.DataFrame

In [15]:
type(train_labels)

pandas.core.series.Series

In [32]:
# create RecordSet of training data
train_features_np = train_features.astype('float32').to_numpy()
train_labels_np = train_labels.astype('float32').to_numpy()

In [33]:
train_labels_np

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [34]:
formatted_train_data = linear.record_set(train_features_np,
                                         labels = train_labels_np,
                                         channel='train',
                                         encrypt=False)

### EXERCISE: Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`, passing in the formatted training data.

In [35]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-10 11:48:36 Starting - Starting the training job...
2021-07-10 11:48:38 Starting - Launching requested ML instancesProfilerReport-1625917716: InProgress
...
2021-07-10 11:49:33 Starting - Preparing the instances for training.........
2021-07-10 11:51:05 Downloading - Downloading input data...
2021-07-10 11:51:35 Training - Downloading the training image...
2021-07-10 11:51:56 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/10/2021 11:52:00 INFO 140687120037696] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': '

[2021-07-10 11:52:16.130] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7236, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1625917936.1308463, "EndTime": 1625917936.1309202, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.016883339575187645, "count": 1, "min": 0.016883339575187645, "max": 0.016883339575187645}}}

#metrics {"StartTime": 1625917936.131014, "EndTime": 1625917936.131029, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014893327607581363, "count": 1, "min": 0.014893327607581363, "max": 0.014893327607581363}}}

#metrics {"StartTime": 1625917936.1310713, "EndTime": 1625917936.131082, "Dimensions": {"Algorithm": "Linear L

[2021-07-10 11:52:23.205] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 7063, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1625917943.2056828, "EndTime": 1625917943.205768, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.010249469512671083, "count": 1, "min": 0.010249469512671083, "max": 0.010249469512671083}}}

#metrics {"StartTime": 1625917943.20589, "EndTime": 1625917943.2059102, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008671593309047832, "count": 1, "min": 0.008671593309047832, "max": 0.008671593309047832}}}

#metrics {"StartTime": 1625917943.205999, "EndTime": 1625917943.2060173, "Dimensions": {"Algorithm": "Linear Le

[2021-07-10 11:52:36.554] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6566, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1625917956.5548134, "EndTime": 1625917956.5548959, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006847595499987578, "count": 1, "min": 0.006847595499987578, "max": 0.006847595499987578}}}

#metrics {"StartTime": 1625917956.5549893, "EndTime": 1625917956.5550084, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005652893664249823, "count": 1, "min": 0.005652893664249823, "max": 0.005652893664249823}}}

#metrics {"StartTime": 1625917956.5550625, "EndTime": 1625917956.5550776, "Dimensions": {"Algorithm": "Line

[2021-07-10 11:52:42.848] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 6282, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1625917962.8487337, "EndTime": 1625917962.8488193, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006208683869347501, "count": 1, "min": 0.006208683869347501, "max": 0.006208683869347501}}}

#metrics {"StartTime": 1625917962.848915, "EndTime": 1625917962.8489358, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005129156069539899, "count": 1, "min": 0.005129156069539899, "max": 0.005129156069539899}}}

#metrics {"StartTime": 1625917962.8489904, "EndTime": 1625917962.8490033, "Dimensions": {"Algorithm": "Linea

[2021-07-10 11:52:57.017] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 17, "duration": 7574, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1625917977.0177677, "EndTime": 1625917977.017845, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 7, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005524270125968972, "count": 1, "min": 0.005524270125968972, "max": 0.005524270125968972}}}

#metrics {"StartTime": 1625917977.0179193, "EndTime": 1625917977.0179372, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 7, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004604272932263475, "count": 1, "min": 0.004604272932263475, "max": 0.004604272932263475}}}

#metrics {"StartTime": 1625917977.0179906, "EndTime": 1625917977.0180063, "Dimensions": {"Algorithm": "Linea

[2021-07-10 11:53:04.121] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 19, "duration": 7091, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1625917984.1214955, "EndTime": 1625917984.1215823, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005324880617946836, "count": 1, "min": 0.005324880617946836, "max": 0.005324880617946836}}}

#metrics {"StartTime": 1625917984.121671, "EndTime": 1625917984.1216915, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004463636301869723, "count": 1, "min": 0.004463636301869723, "max": 0.004463636301869723}}}

#metrics {"StartTime": 1625917984.1217573, "EndTime": 1625917984.1217766, "Dimensions": {"Algorithm": "Linea


2021-07-10 11:53:36 Uploading - Uploading generated training model
2021-07-10 11:53:36 Completed - Training job completed
ProfilerReport-1625917716: NoIssuesFound
Training seconds: 136
Billable seconds: 136
CPU times: user 727 ms, sys: 45.4 ms, total: 773 ms
Wall time: 5min 14s


### EXERCISE: Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [36]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count = 1,
                                 instance_type = 'ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------------------!CPU times: user 377 ms, sys: 22 ms, total: 399 ms
Wall time: 11min 33s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

According to the deployed [predictor documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor), this predictor expects an `ndarray` of input features and returns a list of Records.
> "The prediction is stored in the "predicted_label" key of the `Record.label` field."

Let's first test our model on just one test point, to see the resulting list.

In [40]:
linear_predictor

In [42]:
# # test one prediction
# test_x_np = test_features.astype('float32')
# result = linear_predictor.predict(test_x_np[0])

# print(result)

# test one prediction
test_x_np = test_features.astype('float32').to_numpy()
result = linear_predictor.predict(test_x_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.0002565078029874712
    }
  }
}
]


In [59]:
result[0].label['predicted_label']

float32_tensor {
  values: 0.0
}

### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [60]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [63]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.astype('float32').to_numpy(), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85281   27
1                    36   99

Recall:     0.733
Precision:  0.786
Accuracy:   0.999



## Delete the Endpoint

I've added a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [64]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [65]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-10-11-53-50-335


---

# Model Improvements

The default LinearLearner got a high accuracy, but still classified fraudulent and valid data points incorrectly. Specifically classifying more than 30 points as false negatives (incorrectly labeled, fraudulent transactions), and a little over 30 points as false positives (incorrectly labeled, valid transactions). Let's think about what, during training, could cause this behavior and what we could improve.

**1. Model optimization**
* If we imagine that we are designing this model for use in a bank application, we know that users do *not* want any valid transactions to be categorized as fraudulent. That is, we want to have as few **false positives** (0s classified as 1s) as possible. 
* On the other hand, if our bank manager asks for an application that will catch almost *all* cases of fraud, even if it means a higher number of false positives, then we'd want as few **false negatives** as possible.
* To train according to specific product demands and goals, we do not want to optimize for accuracy only. Instead, we want to optimize for a metric that can help us decrease the number of false positives or negatives. 

<img src='notebook_ims/precision_recall.png' width=40% />
     
In this notebook, we'll look at different cases for tuning a model and make an optimization decision, accordingly.

**2. Imbalanced training data**
* At the start of this notebook, we saw that only about 0.17% of the training data was labeled as fraudulent. So, even if a model labels **all** of our data as valid, it will still have a high accuracy. 
* This may result in some overfitting towards valid data, which accounts for some **false negatives**; cases in which fraudulent data (1) is incorrectly characterized as valid (0).

So, let's address these issues in order; first, tuning our model and optimizing for a specific metric during training, and second, accounting for class imbalance in the training set. 


## Improvement: Model Tuning

Optimizing according to a specific metric is called **model tuning**, and SageMaker provides a number of ways to automatically tune a model.


### Create a LinearLearner and tune for higher precision 

**Scenario:**
* A bank has asked you to build a model that detects cases of fraud with an accuracy of about 85%. 

In this case, we want to build a model that has as many true positives and as few false negatives, as possible. This corresponds to a model with a high **recall**: true positives / (true positives + false negatives). 

To aim for a specific metric, LinearLearner offers the hyperparameter `binary_classifier_model_selection_criteria`, which is the model evaluation criteria for the training dataset. A reference to this parameter is in [LinearLearner's documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner). We'll also have to further specify the exact value we want to aim for; read more about the details of the parameters, [here](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html).

I will assume that performance on a training set will be within about 5% of the performance on a test set. So, for a recall of about 85%, I'll aim for a bit higher, 90%.

In [66]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train the tuned estimator

Fit the new, tuned estimator on the formatted training data.

In [75]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-12 09:45:01 Starting - Starting the training job...
2021-07-12 09:45:03 Starting - Launching requested ML instancesProfilerReport-1626083101: InProgress
......
2021-07-12 09:46:27 Starting - Preparing the instances for training............
2021-07-12 09:48:29 Downloading - Downloading input data
2021-07-12 09:48:29 Training - Downloading the training image...
2021-07-12 09:48:49 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/12/2021 09:48:56 INFO 139847077005120] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

[2021-07-12 09:49:11.560] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7792, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083351.5607615, "EndTime": 1626083351.5608547, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.016883339575187645, "count": 1, "min": 0.016883339575187645, "max": 0.016883339575187645}}}

#metrics {"StartTime": 1626083351.5609646, "EndTime": 1626083351.5610063, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014893327607581363, "count": 1, "min": 0.014893327607581363, "max": 0.014893327607581363}}}

#metrics {"StartTime": 1626083351.561066, "EndTime": 1626083351.5610838, "Dimensions": {"Algorithm": "Linear

[2021-07-12 09:49:17.790] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6217, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083357.7903173, "EndTime": 1626083357.7904122, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.010249469512671083, "count": 1, "min": 0.010249469512671083, "max": 0.010249469512671083}}}

#metrics {"StartTime": 1626083357.7905407, "EndTime": 1626083357.7905629, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008671593309047832, "count": 1, "min": 0.008671593309047832, "max": 0.008671593309047832}}}

#metrics {"StartTime": 1626083357.7906337, "EndTime": 1626083357.7906516, "Dimensions": {"Algorithm": "Linea

[2021-07-12 09:49:30.839] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6743, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083370.839861, "EndTime": 1626083370.8399575, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006847595499987578, "count": 1, "min": 0.006847595499987578, "max": 0.006847595499987578}}}

#metrics {"StartTime": 1626083370.8400466, "EndTime": 1626083370.8400679, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005652893664249823, "count": 1, "min": 0.005652893664249823, "max": 0.005652893664249823}}}

#metrics {"StartTime": 1626083370.840124, "EndTime": 1626083370.8401425, "Dimensions": {"Algorithm": "Linear

...

[2021-07-12 09:50:04.621] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 7234, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083404.6212835, "EndTime": 1626083404.6213932, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005176234991706196, "count": 1, "min": 0.005176234991706196, "max": 0.005176234991706196}}}

#metrics {"StartTime": 1626083404.6215134, "EndTime": 1626083404.6215365, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004362526129538091, "count": 1, "min": 0.004362526129538091, "max": 0.004362526129538091}}}

#metrics {"StartTime": 1626083404.6216044, "EndTime": 1626083404.6216226, "Dimensions": {"Algorithm": "Line

[2021-07-12 09:50:11.457] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6817, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083411.4574938, "EndTime": 1626083411.4575803, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0050622782317837275, "count": 1, "min": 0.0050622782317837275, "max": 0.0050622782317837275}}}

#metrics {"StartTime": 1626083411.4576817, "EndTime": 1626083411.4577034, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00428980573787162, "count": 1, "min": 0.00428980573787162, "max": 0.00428980573787162}}}

#metrics {"StartTime": 1626083411.4577608, "EndTime": 1626083411.4577792, "Dimensions": {"Algorithm": "Li

[2021-07-12 09:50:24.756] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6692, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083424.7566297, "EndTime": 1626083424.75673, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004901528771798215, "count": 1, "min": 0.004901528771798215, "max": 0.004901528771798215}}}

#metrics {"StartTime": 1626083424.7568283, "EndTime": 1626083424.7568483, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004192210530515891, "count": 1, "min": 0.004192210530515891, "max": 0.004192210530515891}}}

#metrics {"StartTime": 1626083424.7568986, "EndTime": 1626083424.7569094, "Dimensions": {"Algorithm": "Line

[2021-07-12 09:50:31.203] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6434, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626083431.203722, "EndTime": 1626083431.2039666, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00484367591711744, "count": 1, "min": 0.00484367591711744, "max": 0.00484367591711744}}}

#metrics {"StartTime": 1626083431.2040696, "EndTime": 1626083431.2042084, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0041608323353019795, "count": 1, "min": 0.0041608323353019795, "max": 0.0041608323353019795}}}

#metrics {"StartTime": 1626083431.2042766, "EndTime": 1626083431.20441, "Dimensions": {"Algorithm": "Linea


2021-07-12 09:50:51 Uploading - Uploading generated training model
2021-07-12 09:50:51 Completed - Training job completed
Training seconds: 155
Billable seconds: 155
CPU times: user 852 ms, sys: 47.2 ms, total: 899 ms
Wall time: 6min 14s


### Deploy and evaluate the tuned estimator

Deploy the tuned predictor and evaluate it.

We hypothesized that a tuned model, optimized for a higher recall, would have fewer false negatives (fraudulent transactions incorrectly labeled as valid); did the number of false negatives get reduced after tuning the model?

In [76]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 286 ms, sys: 42.1 ms, total: 328 ms
Wall time: 9min 32s


In [77]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.astype('float32').to_numpy(), 
                   test_labels.astype('float32').to_numpy(), 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0.0               84317  991
1.0                  20  115

Recall:     0.852
Precision:  0.104
Accuracy:   0.988



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [78]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-12-09-51-15-988


---
## Improvement: Managing Class Imbalance

We have a model that is tuned to get a higher recall, which aims to reduce the number of false negatives. Earlier, we discussed how class imbalance may actually bias our model towards predicting that all transactions are valid, resulting in higher false negatives and true negatives. It stands to reason that this model could be further improved if we account for this imbalance.

To account for class imbalance during training of a binary classifier, LinearLearner offers the hyperparameter, `positive_example_weight_mult`, which is the weight assigned to positive (1, fraudulent) examples when training a binary classifier. The weight of negative examples (0, valid) is fixed at 1. 

### EXERCISE: Create a LinearLearner with a `positive_example_weight_mult` parameter

In **addition** to tuning a model for higher recall (you may use `linear_recall` as a starting point), you should *add* a parameter that helps account for class imbalance. From the [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) on `positive_example_weight_mult`, it reads:
> "If you want the algorithm to choose a weight so that errors in classifying negative vs. positive examples have equal impact on training loss, specify `balanced`."

You could also put in a specific float value, in which case you'd want to weight positive examples more heavily than negative examples, since there are fewer of them.

In [80]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                positive_example_weight_mult='balanced',
                                binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                                target_recall=0.9) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### EXERCISE: Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [81]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-12 13:22:48 Starting - Starting the training job...
2021-07-12 13:23:12 Starting - Launching requested ML instancesProfilerReport-1626096168: InProgress
......
2021-07-12 13:24:12 Starting - Preparing the instances for training.........
2021-07-12 13:25:33 Downloading - Downloading input data...
2021-07-12 13:26:17 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/12/2021 13:26:22 INFO 140293672294208] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

[2021-07-12 13:26:37.336] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7395, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096397.3368223, "EndTime": 1626096397.3369014, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3762333395109704, "count": 1, "min": 0.3762333395109704, "max": 0.3762333395109704}}}

#metrics {"StartTime": 1626096397.3369775, "EndTime": 1626096397.3369963, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3452077499849712, "count": 1, "min": 0.3452077499849712, "max": 0.3452077499849712}}}

#metrics {"StartTime": 1626096397.3370545, "EndTime": 1626096397.3370707, "Dimensions": {"Algorithm": 

[2021-07-12 13:26:51.991] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 7327, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096411.991453, "EndTime": 1626096411.9915361, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.32226599174767884, "count": 1, "min": 0.32226599174767884, "max": 0.32226599174767884}}}

#metrics {"StartTime": 1626096411.9916332, "EndTime": 1626096411.9916549, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3074733453491824, "count": 1, "min": 0.3074733453491824, "max": 0.3074733453491824}}}

#metrics {"StartTime": 1626096411.9917085, "EndTime": 1626096411.991725, "Dimensions": {"Algorithm":

[2021-07-12 13:26:58.947] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6942, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096418.9475017, "EndTime": 1626096418.947576, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3127615477978884, "count": 1, "min": 0.3127615477978884, "max": 0.3127615477978884}}}

#metrics {"StartTime": 1626096418.947683, "EndTime": 1626096418.9477036, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.30119617619346734, "count": 1, "min": 0.30119617619346734, "max": 0.30119617619346734}}}

#metrics {"StartTime": 1626096418.9477544, "EndTime": 1626096418.9477708, "Dimensions": {"Algorithm"

...

[2021-07-12 13:27:36.045] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 7341, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096456.0457826, "EndTime": 1626096456.04589, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2953641708987442, "count": 1, "min": 0.2953641708987442, "max": 0.2953641708987442}}}

#metrics {"StartTime": 1626096456.0459585, "EndTime": 1626096456.0459774, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28941299503652296, "count": 1, "min": 0.28941299503652296, "max": 0.28941299503652296}}}

#metrics {"StartTime": 1626096456.0460315, "EndTime": 1626096456.046047, "Dimensions": {"Algorithm":

[2021-07-12 13:27:48.565] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 6082, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096468.5657806, "EndTime": 1626096468.565882, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2928580866674682, "count": 1, "min": 0.2928580866674682, "max": 0.2928580866674682}}}

#metrics {"StartTime": 1626096468.565965, "EndTime": 1626096468.5659854, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2874274079979365, "count": 1, "min": 0.2874274079979365, "max": 0.2874274079979365}}}

#metrics {"StartTime": 1626096468.5660422, "EndTime": 1626096468.5660584, "Dimensions": {"Algorithm":

[2021-07-12 13:28:02.535] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6767, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096482.5357504, "EndTime": 1626096482.5359793, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29126332237972086, "count": 1, "min": 0.29126332237972086, "max": 0.29126332237972086}}}

#metrics {"StartTime": 1626096482.5360758, "EndTime": 1626096482.5360959, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2859753795221223, "count": 1, "min": 0.2859753795221223, "max": 0.2859753795221223}}}

#metrics {"StartTime": 1626096482.536156, "EndTime": 1626096482.5361745, "Dimensions": {"Algorit


2021-07-12 13:28:15 Uploading - Uploading generated training model[2021-07-12 13:28:09.856] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 7307, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626096489.8570325, "EndTime": 1626096489.8571181, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29068308453583835, "count": 1, "min": 0.29068308453583835, "max": 0.29068308453583835}}}

#metrics {"StartTime": 1626096489.857221, "EndTime": 1626096489.8572412, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28538028095954626, "count": 1, "min": 0.28538028095954626, "max": 0.28538028095954626}}}

#metrics {"StartTime": 1626


2021-07-12 13:28:53 Completed - Training job completed
ProfilerReport-1626096168: NoIssuesFound
Training seconds: 180
Billable seconds: 180
CPU times: user 916 ms, sys: 54 ms, total: 970 ms
Wall time: 6min 15s


### EXERCISE: Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [82]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1,
                                            instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 315 ms, sys: 23.7 ms, total: 339 ms
Wall time: 9min 32s


In [83]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.astype('float32').to_numpy(), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 84629  679
1                    18  117

Recall:     0.867
Precision:  0.147
Accuracy:   0.992



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [84]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-12-13-29-04-393


A note on metric variability: 

The above model is tuned for the best possible precision with recall fixed at about 90%. The recall is fixed at 90% during training, but may vary when we apply our trained model to a test set of data.

---
## Model Design

Now that you've seen how to tune and balance a LinearLearner. Create, train and deploy your own model. This exercise is meant to be more open-ended, so that you get practice with the steps involved in designing a model and deploying it.

### EXERCISE: Train and deploy a LinearLearner with appropriate hyperparameters, according to the given scenario

**Scenario:**
* A bank has asked you to build a model that optimizes for a good user experience; users should only ever have up to about 15% of their valid transactions flagged as fraudulent.

This requires that you make a design decision: Given the above scenario, what metric (and value) should you aim for during training?

You may assume that performance on a training set will be within about 5-10% of the performance on a test set. For example, if you get 80% on a training set, you can assume that you'll get between about 70-90% accuracy on a test set.

Your final model should account for class imbalance and be appropriately tuned. 

In [86]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data
linear_precision = LinearLearner(role=role, 
                                 output_path=output_path, 
                                 sagemaker_session=sagemaker_session, 
                                 epochs=15,
                                 train_instance_count=1, 
                                 train_instance_type='ml.c4.xlarge', 
                                 predictor_type='binary_classifier',
                                 positive_example_weight_mult='balanced',
                                 binary_classifier_model_selection_criteria='recall_at_target_precision',
                                 target_precision=0.9)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


CPU times: user 4.59 ms, sys: 3 µs, total: 4.59 ms
Wall time: 4.45 ms


In [88]:
linear_precision.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-12 13:41:49 Starting - Starting the training job...
2021-07-12 13:42:12 Starting - Launching requested ML instancesProfilerReport-1626097309: InProgress
...
2021-07-12 13:42:46 Starting - Preparing the instances for training............
2021-07-12 13:44:42 Downloading - Downloading input data...
2021-07-12 13:45:14 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/12/2021 13:45:19 INFO 140201902622528] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

[2021-07-12 13:45:41.241] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6948, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097541.2416117, "EndTime": 1626097541.2417283, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3391149269755761, "count": 1, "min": 0.3391149269755761, "max": 0.3391149269755761}}}

#metrics {"StartTime": 1626097541.2418308, "EndTime": 1626097541.2418482, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3195404147047493, "count": 1, "min": 0.3195404147047493, "max": 0.3195404147047493}}}

#metrics {"StartTime": 1626097541.2418838, "EndTime": 1626097541.2418928, "Dimensions": {"Algorithm": 

[2021-07-12 13:45:48.803] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 7548, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097548.8037724, "EndTime": 1626097548.803861, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.32226599174767884, "count": 1, "min": 0.32226599174767884, "max": 0.32226599174767884}}}

#metrics {"StartTime": 1626097548.8039424, "EndTime": 1626097548.8039618, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3074733453491824, "count": 1, "min": 0.3074733453491824, "max": 0.3074733453491824}}}

#metrics {"StartTime": 1626097548.8040128, "EndTime": 1626097548.8040285, "Dimensions": {"Algorithm"

[2021-07-12 13:46:02.941] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 7250, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097562.9418418, "EndTime": 1626097562.9419389, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.30667649917985923, "count": 1, "min": 0.30667649917985923, "max": 0.30667649917985923}}}

#metrics {"StartTime": 1626097562.942041, "EndTime": 1626097562.9420598, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29720698666213147, "count": 1, "min": 0.29720698666213147, "max": 0.29720698666213147}}}

#metrics {"StartTime": 1626097562.942105, "EndTime": 1626097562.9421177, "Dimensions": {"Algorit

...

[2021-07-12 13:46:31.389] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6818, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097591.3893268, "EndTime": 1626097591.3895392, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2953641708987442, "count": 1, "min": 0.2953641708987442, "max": 0.2953641708987442}}}

#metrics {"StartTime": 1626097591.3896391, "EndTime": 1626097591.3896825, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28941299503652296, "count": 1, "min": 0.28941299503652296, "max": 0.28941299503652296}}}

#metrics {"StartTime": 1626097591.3897507, "EndTime": 1626097591.3897681, "Dimensions": {"Algorith

[2021-07-12 13:46:38.583] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 7179, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097598.58373, "EndTime": 1626097598.5838153, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2939692262045702, "count": 1, "min": 0.2939692262045702, "max": 0.2939692262045702}}}

#metrics {"StartTime": 1626097598.583909, "EndTime": 1626097598.5839307, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28833337498190414, "count": 1, "min": 0.28833337498190414, "max": 0.28833337498190414}}}

#metrics {"StartTime": 1626097598.5839696, "EndTime": 1626097598.5839794, "Dimensions": {"Algorithm

[2021-07-12 13:46:51.889] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6353, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097611.8892374, "EndTime": 1626097611.889455, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2919692740128867, "count": 1, "min": 0.2919692740128867, "max": 0.2919692740128867}}}

#metrics {"StartTime": 1626097611.8895495, "EndTime": 1626097611.8895717, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2866512928488267, "count": 1, "min": 0.2866512928488267, "max": 0.2866512928488267}}}

#metrics {"StartTime": 1626097611.889631, "EndTime": 1626097611.8896794, "Dimensions": {"Algorithm":

[2021-07-12 13:46:58.614] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6710, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626097618.6140997, "EndTime": 1626097618.6141975, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29126332237972086, "count": 1, "min": 0.29126332237972086, "max": 0.29126332237972086}}}

#metrics {"StartTime": 1626097618.6142993, "EndTime": 1626097618.6143143, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2859753795221223, "count": 1, "min": 0.2859753795221223, "max": 0.2859753795221223}}}

#metrics {"StartTime": 1626097618.614346, "EndTime": 1626097618.6143546, "Dimensions": {"Algorit


2021-07-12 13:47:21 Uploading - Uploading generated training model
2021-07-12 13:47:21 Completed - Training job completed
Training seconds: 159
Billable seconds: 159


In [89]:
%%time 
# deploy and evaluate a predictor
precision_predictor = linear_precision.deploy(initial_instance_count=1, 
                                              instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------------!CPU times: user 557 ms, sys: 4.9 ms, total: 562 ms
Wall time: 10min 33s


In [92]:
print('Metrics for higher precision, LinearLearner.\n')

# get metrics for balanced predictor
precision_metrics = evaluate(precision_predictor, 
                             test_features.astype('float32').to_numpy(), 
                             test_labels, 
                             verbose=True)

Metrics for higher precision, LinearLearner.

prediction (col)  0.0    1.0
actual (row)                
0                 929  84379
1                   0    135

Recall:     1.000
Precision:  0.002
Accuracy:   0.012



In [93]:
## IMPORTANT
# delete the predictor endpoint after evaluation 
delete_endpoint(precision_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-12-13-47-33-600


# Test Udacity's code

In [94]:
%%time

linear_precision = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='recall_at_target_precision',
                                target_precision=0.9,
                                positive_example_weight_mult='balanced')

linear_precision.fit(formatted_train_data)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-12 14:05:57 Starting - Starting the training job...
2021-07-12 14:06:21 Starting - Launching requested ML instancesProfilerReport-1626098757: InProgress
......
2021-07-12 14:07:21 Starting - Preparing the instances for training............
2021-07-12 14:09:28 Downloading - Downloading input data
2021-07-12 14:09:28 Training - Downloading the training image...
2021-07-12 14:09:49 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/12/2021 14:09:55 INFO 140326639097664] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

[2021-07-12 14:10:11.161] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 8080, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626099011.1611412, "EndTime": 1626099011.161223, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3762333395109704, "count": 1, "min": 0.3762333395109704, "max": 0.3762333395109704}}}

#metrics {"StartTime": 1626099011.1613111, "EndTime": 1626099011.1613307, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3452077499849712, "count": 1, "min": 0.3452077499849712, "max": 0.3452077499849712}}}

#metrics {"StartTime": 1626099011.1613855, "EndTime": 1626099011.1614022, "Dimensions": {"Algorithm": "

[2021-07-12 14:10:18.069] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6895, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626099018.0696962, "EndTime": 1626099018.0697691, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3391149269755761, "count": 1, "min": 0.3391149269755761, "max": 0.3391149269755761}}}

#metrics {"StartTime": 1626099018.0698514, "EndTime": 1626099018.0698664, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3195404147047493, "count": 1, "min": 0.3195404147047493, "max": 0.3195404147047493}}}

#metrics {"StartTime": 1626099018.0699108, "EndTime": 1626099018.0699215, "Dimensions": {"Algorithm": 

[2021-07-12 14:10:24.957] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6875, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626099024.95747, "EndTime": 1626099024.9575503, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.32226599174767884, "count": 1, "min": 0.32226599174767884, "max": 0.32226599174767884}}}

#metrics {"StartTime": 1626099024.9576387, "EndTime": 1626099024.9576573, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3074733453491824, "count": 1, "min": 0.3074733453491824, "max": 0.3074733453491824}}}

#metrics {"StartTime": 1626099024.9577093, "EndTime": 1626099024.9577255, "Dimensions": {"Algorithm":

...

[2021-07-12 14:11:16.202] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6947, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626099076.2023504, "EndTime": 1626099076.202542, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2939692262045702, "count": 1, "min": 0.2939692262045702, "max": 0.2939692262045702}}}

#metrics {"StartTime": 1626099076.2026312, "EndTime": 1626099076.2026482, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.28833337498190414, "count": 1, "min": 0.28833337498190414, "max": 0.28833337498190414}}}

#metrics {"StartTime": 1626099076.2027113, "EndTime": 1626099076.2027287, "Dimensions": {"Algorit

[2021-07-12 14:11:29.989] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6710, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626099089.9894233, "EndTime": 1626099089.9895086, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2919692740128867, "count": 1, "min": 0.2919692740128867, "max": 0.2919692740128867}}}

#metrics {"StartTime": 1626099089.9896066, "EndTime": 1626099089.9896278, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2866512928488267, "count": 1, "min": 0.2866512928488267, "max": 0.2866512928488267}}}

#metrics {"StartTime": 1626099089.9896872, "EndTime": 1626099089.9897063, "Dimensions": {"Algorithm

[2021-07-12 14:11:37.462] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 7459, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626099097.4620802, "EndTime": 1626099097.4621649, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.29126332237972086, "count": 1, "min": 0.29126332237972086, "max": 0.29126332237972086}}}

#metrics {"StartTime": 1626099097.4622302, "EndTime": 1626099097.4622476, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.2859753795221223, "count": 1, "min": 0.2859753795221223, "max": 0.2859753795221223}}}

#metrics {"StartTime": 1626099097.4623036, "EndTime": 1626099097.462322, "Dimensions": {"Algorit


2021-07-12 14:11:58 Uploading - Uploading generated training model
2021-07-12 14:11:58 Completed - Training job completed
Training seconds: 162
Billable seconds: 162


UsageError: Line magic function `%%time` not found.


In [96]:
%%time 
# deploy and evaluate a predictor
precision_predictor = linear_precision.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 320 ms, sys: 18.5 ms, total: 338 ms
Wall time: 9min 33s


In [100]:
print('Metrics for precision_predictor, LinearLearner.\n')
metrics = evaluate(precision_predictor, 
                   test_features.astype('float32').to_numpy(), 
                   test_labels, 
                   verbose=True)

Metrics for precision_predictor, LinearLearner.

prediction (col)  0.0    1.0
actual (row)                
0                 929  84379
1                   0    135

Recall:     1.000
Precision:  0.002
Accuracy:   0.012



In [104]:
delete_endpoint(precision_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-12-14-14-48-091


In [102]:
# Without balance 
%%time

linear_precision_no_bal = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='recall_at_target_precision',
                                target_precision=0.9)

linear_precision_no_bal.fit(formatted_train_data)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-12 14:33:44 Starting - Starting the training job...
2021-07-12 14:34:07 Starting - Launching requested ML instancesProfilerReport-1626100423: InProgress
...
2021-07-12 14:34:40 Starting - Preparing the instances for training............
2021-07-12 14:36:42 Downloading - Downloading input data
2021-07-12 14:36:42 Training - Downloading the training image...
2021-07-12 14:37:08 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/12/2021 14:37:08 INFO 139971275704128] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': '

[2021-07-12 14:37:23.352] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7325, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100643.3525105, "EndTime": 1626100643.3527455, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.016883339575187645, "count": 1, "min": 0.016883339575187645, "max": 0.016883339575187645}}}

#metrics {"StartTime": 1626100643.3528287, "EndTime": 1626100643.3528497, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014893327607581363, "count": 1, "min": 0.014893327607581363, "max": 0.014893327607581363}}}

#metrics {"StartTime": 1626100643.3529296, "EndTime": 1626100643.3529491, "Dimensions": {"Algorithm": "Linea

[2021-07-12 14:37:30.133] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6767, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100650.1335568, "EndTime": 1626100650.133641, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.010249469512671083, "count": 1, "min": 0.010249469512671083, "max": 0.010249469512671083}}}

#metrics {"StartTime": 1626100650.1337883, "EndTime": 1626100650.1338112, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008671593309047832, "count": 1, "min": 0.008671593309047832, "max": 0.008671593309047832}}}

#metrics {"StartTime": 1626100650.133864, "EndTime": 1626100650.1338806, "Dimensions": {"Algorithm": "Linear 

[2021-07-12 14:37:43.452] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6809, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100663.452331, "EndTime": 1626100663.4525495, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006847595499987578, "count": 1, "min": 0.006847595499987578, "max": 0.006847595499987578}}}

#metrics {"StartTime": 1626100663.4526412, "EndTime": 1626100663.4526558, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005652893664249823, "count": 1, "min": 0.005652893664249823, "max": 0.005652893664249823}}}

#metrics {"StartTime": 1626100663.4527178, "EndTime": 1626100663.4527345, "Dimensions": {"Algorithm": "Linea

...

[2021-07-12 14:38:11.244] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 19, "duration": 7321, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100691.2449658, "EndTime": 1626100691.245054, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005324880617946836, "count": 1, "min": 0.005324880617946836, "max": 0.005324880617946836}}}

#metrics {"StartTime": 1626100691.2451453, "EndTime": 1626100691.245161, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 8, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004463636301869723, "count": 1, "min": 0.004463636301869723, "max": 0.004463636301869723}}}

#metrics {"StartTime": 1626100691.2452073, "EndTime": 1626100691.2452192, "Dimensions": {"Algorithm": "Linear

[2021-07-12 14:38:24.721] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6751, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100704.7216177, "EndTime": 1626100704.7217088, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0050622782317837275, "count": 1, "min": 0.0050622782317837275, "max": 0.0050622782317837275}}}

#metrics {"StartTime": 1626100704.7218077, "EndTime": 1626100704.72183, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00428980573787162, "count": 1, "min": 0.00428980573787162, "max": 0.00428980573787162}}}

#metrics {"StartTime": 1626100704.7218676, "EndTime": 1626100704.7218776, "Dimensions": {"Algorithm": "Line

[2021-07-12 14:38:37.966] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6668, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100717.9668367, "EndTime": 1626100717.9669297, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004901528771798215, "count": 1, "min": 0.004901528771798215, "max": 0.004901528771798215}}}

#metrics {"StartTime": 1626100717.9670107, "EndTime": 1626100717.9670324, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004192210530515891, "count": 1, "min": 0.004192210530515891, "max": 0.004192210530515891}}}

#metrics {"StartTime": 1626100717.9670885, "EndTime": 1626100717.9671047, "Dimensions": {"Algorithm": "Li

[2021-07-12 14:38:45.123] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 7145, "num_examples": 200, "num_bytes": 33493152}
#metrics {"StartTime": 1626100725.1234286, "EndTime": 1626100725.123512, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.00484367591711744, "count": 1, "min": 0.00484367591711744, "max": 0.00484367591711744}}}

#metrics {"StartTime": 1626100725.123608, "EndTime": 1626100725.12363, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0041608323353019795, "count": 1, "min": 0.0041608323353019795, "max": 0.0041608323353019795}}}

#metrics {"StartTime": 1626100725.123688, "EndTime": 1626100725.1237044, "Dimensions": {"Algorithm": "Linear 


2021-07-12 14:39:08 Uploading - Uploading generated training model
2021-07-12 14:39:08 Completed - Training job completed
ProfilerReport-1626100423: NoIssuesFound
Training seconds: 148
Billable seconds: 148
CPU times: user 833 ms, sys: 43.8 ms, total: 877 ms
Wall time: 5min 44s


In [103]:
%%time 

precision_no_bal_predictor = linear_precision_no_bal.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

print('Metrics for precision_no_bal_predictor, LinearLearner.\n')

metrics = evaluate(precision_no_bal_predictor, 
                   test_features.astype('float32').to_numpy(), 
                   test_labels, 
                   verbose=True)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------------!Metrics for precision_no_bal_predictor, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85307    1
1                    97   38

Recall:     0.281
Precision:  0.974
Accuracy:   0.999

CPU times: user 5.4 s, sys: 62.7 ms, total: 5.46 s
Wall time: 10min 49s


In [105]:
delete_endpoint(precision_no_bal_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-07-12-14-39-28-326


## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a LinearLearner in SageMaker. This model is well-suited for a binary classification task that involves specific design decisions and managing class imbalance in the training set.

Following the steps of a machine learning workflow, you loaded in some credit card transaction data, explored that data and prepared it for model training. Then trained, deployed, and evaluated several models, according to different design considerations!